In [60]:
import os
from sentence_transformers import SentenceTransformer, util

In [ ]:
# ! pip install bert-score


In [ ]:
#! pip install sentence-transformers


In [ ]:
# ! pip install torch==1.12.1 torchvision torchaudio


In [44]:
os.chdir('..')

In [61]:
# Change directory
# os.chdir('AlignScore')

# Verify current working directory
print(os.getcwd())

/Users/osultan/Documents/01-PhD/01-Studying/08-Fall_24/F3 Project/Trial 3/F3 Codebase/Datasets/Co-AidPost_GTP3.5_Real_Posts_Output_Data


In [ ]:
# ! pip install .

In [48]:
# os.chdir('..')
os.chdir('X-GenPost_GTP3.5_Real_Posts_Output_Data')

In [ ]:
# Install spacy
# !pip install spacy

# Download the required spaCy model
# !python -m spacy download en_core_web_sm


In [49]:
from alignscore import AlignScore
import pandas as pd
import spacy


In [ ]:

#print(spacy.info())



{'spacy_version': '3.7.5', 'location': '/Users/osultan/anaconda3/lib/python3.10/site-packages/spacy', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'python_version': '3.10.10', 'pipelines': {}}


In [ ]:
# ! pip install spacy
#!python -m spacy download en_core_web_sm
#! which python

In [ ]:
#!/Users/osultan/anaconda3/bin/pip install spacy
#!/Users/osultan/anaconda3/bin/python -m spacy download en_core_web_sm


In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")
print("spaCy model loaded successfully!")


spaCy model loaded successfully!


In [4]:
print(spacy.info()['pipelines'])

{'en_core_web_sm': '3.7.1'}


In [67]:
# Load your DataFrame
df = pd.read_csv("Critical_articles_1-100.csv")
df.head()

,uuid,human_written_content,aigenerated_content,model,num_completion_token,num_original_token,num_prompt_token,num_iagenerated_token,original_label,source_type,ai_generated_label,article_type,pre_post_GPT,dataset_source
0,a6477508-e98c-4d3b-bb5f-04521a092609,as of todays reports the global number of conf...,The World Health Organization (WHO) has highli...,gpt-3.5-turbo-0125,56,80,175,231,real,AI Machine,real,news article,pre-GPT,CoAID
1,60343d09-529c-45f9-a157-d95cfb70ee83,"""And they have almost the same numbers. What i...",Michigan has a significantly higher percentage...,gpt-3.5-turbo-0125,43,64,158,201,real,AI Machine,real,twitter post,pre-GPT,CoAID
2,061da836-d0cf-4418-afa0-cbbd2cae37e0,a new poll found that 58 percent of respondent...,A recent survey revealed that 58% of participa...,gpt-3.5-turbo-0125,75,92,187,262,real,AI Machine,real,news article,pre-GPT,CoAID
3,1243e23f-c38e-43ca-8fa3-99c07eaa549e,"""Whatever happened to Cummings & Co's herd imm...",Exploring the status of Cummings & Co's herd i...,gpt-3.5-turbo-0125,34,26,127,161,real,AI Machine,real,twitter post,pre-GPT,CoAID
4,3e696b1f-49ac-4571-87e0-d528b5ef9428,scientists have found that four young males wh...,A recent study revealed that four young Dutch ...,gpt-3.5-turbo-0125,56,80,174,230,real,AI Machine,real,news article,pre-GPT,CoAID


In [51]:
# Initialize AlignScore
scorer = AlignScore(
    model='roberta-base',                # Model backbone
    batch_size=32,                       # Batch size
    device='cpu',                        # Change to 'cpu' if no GPU is available
    ckpt_path='/Users/osultan/Documents/01-PhD/01-Studying/08-Fall_24/F3 Project/Trial 3/F3 Codebase/AlignScore-base.ckpt',    # Path to the checkpoint file
    evaluation_mode='nli_sp'             # Default evaluation mode
)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Lightning automatically upgraded your loaded checkpoint from v1.7.7 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../AlignScore-base.ckpt`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/osultan/anaconda3/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['base_model.embed

In [ ]:
# import os

# # Path to the checkpoint
# ckpt_path = 'AlignScore-base.ckpt'

# # Check if the file exists
# if os.path.exists(ckpt_path):
#     print(f"Checkpoint file found at: {ckpt_path}")
# else:
#     print(f"Checkpoint file not found at: {ckpt_path}. Please verify the path.")


Checkpoint file found at: AlignScore-base.ckpt


In [ ]:
#score = scorer.score(contexts=['hello world.'], claims=['hello world.'])

Evaluating: 100%|██████████| 1/1 [00:03<00:00,  3.63s/it]


In [ ]:
#score

[0.9947293400764465]

In [52]:
# Function to calculate AlignScore
def calculate_alignscore(human_texts, ai_texts):
    """
    Calculate AlignScore for a list of human-written and AI-generated texts.
    """
    try:
        scores = scorer.score(contexts=human_texts, claims=ai_texts)
    except Exception as e:
        scores = [f"Error: {str(e)}"] * len(human_texts)
    return scores



In [68]:
# Prepare texts for batch processing
batch_size = 100
align_scores = []

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    human_texts = batch["human_written_content"].tolist()
    ai_texts = batch["aigenerated_content"].tolist()
    batch_scores = calculate_alignscore(human_texts, ai_texts)
    align_scores.extend(batch_scores)

# Add AlignScore to the DataFrame
df["AlignScore"] = align_scores

# Save results to a new CSV file
df.to_csv("alignscore_results.csv", index=False)

print("AlignScore calculation completed. Results saved to alignscore_results.csv")

Evaluating: 100%|██████████| 100/100 [18:41<00:00, 11.21s/it]

AlignScore calculation completed. Results saved to alignscore_results.csv


In [54]:
import pandas as pd
from bert_score import score


In [55]:
# Function to calculate BERTScore
def calculate_bertscore(human_text, ai_text, model_type="roberta-large"):
    """
    Calculate BERTScore for contextual alignment.

    Args:
        human_text (str): The human-written content.
        ai_text (str): The AI-generated content.
        model_type (str): The pretrained model to use for BERTScore (default is 'roberta-large').

    Returns:
        dict: Precision, Recall, and F1 BERTScore.
    """
    try:
        # Calculate BERTScore
        P, R, F1 = score([ai_text], [human_text], model_type=model_type, lang="en", verbose=True)
        return {
            "BERTScore_Precision": P.item(),
            "BERTScore_Recall": R.item(),
            "BERTScore_F1": F1.item()
        }
    except Exception as e:
        return {
            "Error": str(e)
        }



In [ ]:
# Iterate over DataFrame and calculate BERTScore for each row
results = []
for index, row in df.iterrows():
    human_text = row["human_written_content"]
    ai_text = row["aigenerated_content"]
    bertscore_result = calculate_bertscore(human_text, ai_text)
    results.append({
        "human_written_content": human_text,
        "ai_generated_content": ai_text,
        **bertscore_result
    })

# Save results to a new DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv("bertscore_results.csv", index=False)

print("BERTScore results have been saved to 'bertscore_results.csv'.")


In [65]:
# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose other models based on accuracy vs. speed



In [58]:
# Function to calculate semantic distance
def calculate_semantic_distance(human_text, ai_text):
    """
    Calculate semantic distance between human-written and AI-generated content.

    Args:
        human_text (str): The human-written content.
        ai_text (str): The AI-generated content.

    Returns:
        float: The semantic distance between the two texts.
    """
    try:
        # Get embeddings for both texts
        human_embedding = model.encode(human_text, convert_to_tensor=True)
        ai_embedding = model.encode(ai_text, convert_to_tensor=True)

        # Calculate cosine similarity
        similarity = util.cos_sim(human_embedding, ai_embedding).item()

        # Calculate semantic distance (1 - similarity)
        distance = 1 - similarity
        return distance
    except Exception as e:
        return {"Error": str(e)}



In [70]:
# Iterate over DataFrame and calculate semantic distance for each row
results = []
for index, row in df.iterrows():
    human_text = row["human_written_content"]
    ai_text = row["aigenerated_content"]
    semantic_distance = calculate_semantic_distance(human_text, ai_text)
    results.append({
        "human_written_content": human_text,
        "ai_generated_content": ai_text,
        "Semantic_Distance": semantic_distance
    })

# Save results to a new DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv("semantic_distance_results.csv", index=False)

print("Semantic Distance results have been saved to 'semantic_distance_results.csv'.")

Semantic Distance results have been saved to 'semantic_distance_results.csv'.
